It seems the current [high scoring script][1] is written in R using H2O. So let us do one in python using XGBoost. 

Thanks to [this script][2] for feature engineering ideas. 

We shall start with importing the necessary modules


  [1]: https://www.kaggle.com/gospursgo/two-sigma-connect-rental-listing-inquiries/h2o-starter-pack/run/835757
  [2]: https://www.kaggle.com/aikinogard/two-sigma-connect-rental-listing-inquiries/random-forest-starter-with-numerical-features

In [ ]:
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

import matplotlib.pyplot as plt

from xgboost.sklearn import XGBClassifier

Now let us write a custom function to run the xgboost model.

Let us read the train and test files and store it.

In [ ]:
data_path = "../input/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)

In [ ]:
train_df.columns

In [ ]:
train_df.head()

In [ ]:
train_df.info()

We do not need any pre-processing for numerical features and so create a list with those features.

In [ ]:
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]

Now let us create some new features from the given features.

In [ ]:
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

# adding all these new features to use list #
features_to_use.extend(["num_photos", "num_features", "num_description_words","created_year", "created_month", "created_day", "listing_id", "created_hour"])

We have 4 categorical features in our data

 - display_address
 - manager_id
 - building_id
 - listing_id

So let us label encode these features.

In [ ]:
categorical = ["display_address", "manager_id", "building_id", "street_address"]
for f in categorical:
        if train_df[f].dtype=='object':
            #print(f)
            lbl = preprocessing.LabelEncoder()
            lbl.fit(list(train_df[f].values) + list(test_df[f].values))
            train_df[f] = lbl.transform(list(train_df[f].values))
            test_df[f] = lbl.transform(list(test_df[f].values))
            features_to_use.append(f)

We have features column which is a list of string values. So we can first combine all the strings together to get a single string and then apply count vectorizer on top of it.

In [ ]:
train_df['features'] = train_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
test_df['features'] = test_df["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))
print(train_df["features"].head())
tfidf = CountVectorizer(stop_words='english', max_features=200)
tr_sparse = tfidf.fit_transform(train_df["features"])
te_sparse = tfidf.transform(test_df["features"])

In [ ]:
train_df.head()

Now let us stack both the dense and sparse features into a single dataset and also get the target variable.

In [ ]:
train_df[features_to_use].head()

In [ ]:
train_X = train_df[features_to_use]
test_X = test_df[features_to_use]

target_num_map = {'high':0, 'medium':1, 'low':2}
train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))
print(train_X.shape, test_X.shape)

In [ ]:
train_X

In [ ]:
train_y

In [ ]:
train_df['interest_level']

Now let us do some cross validation to check the scores. 

Please run it in local to get the cv scores. I am commenting it out here for time.

In [ ]:
features_to_use

In [ ]:
def modelfit(alg, X,y, useTrainCV=True,
             cv_fold=5,early_stopping_rounds = 20):
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgb_param['num_class']=3
        xgtrain = xgb.DMatrix(X,label = y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=
                          alg.get_params()['n_estimators'],nfold=cv_fold,
                          metrics='mlogloss',early_stopping_rounds = early_stopping_rounds)
        print(cvresult)
        alg.set_params(n_estimators=cvresult.shape[0])
        
    #Fit the algorithm on the data
    alg.fit(X, y,eval_metric='mlogloss')
        
    #Predict training set:
    y_pred = alg.predict(X)
    y_predprob = alg.predict_proba(X)
    print(y_predprob)
        
    #Pring model report
    print ("\nModel Report")
    #print ("Accuracy : %.4g" % accuracy_score(y, 
                                            #y_pred))
    print ("logloss score (Train) : %f" % log_loss(y,  y_predprob))
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='hbar',title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
xgb1 = XGBClassifier(
    learning_rate = 0.1,
    n_estimators=100,
    max_depth=5,
    min_child_weight=1,
    gamma = 0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='multi:softprob',
    nthread =4,
    scale_pos_weight=1,
    seed = 0,
)

In [ ]:
modelfit(xgb1, train_X, train_y)

In [ ]:
xgb1

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_test1={
    'max_depth':range(1,10,2),
    'min_child_weight':range(1,10,3)
}

gs1 = GridSearchCV(xgb1,param_grid=param_test1, 
                   scoring='neg_log_loss', n_jobs=-1,iid=False, cv=5)
gs1.fit(train_X,train_y)
gs1.grid_scores_, gs1.best_params_,gs1.best_score_

Now let us build the final model and get the predictions on the test set.

In [ ]:
preds, model = runXGB(train_X, train_y, test_X, num_rounds=400)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
#out_df.to_csv("xgb2.csv", index=False)
out_df.head()


Hope this helps the python users as a good starting point.